<a href="https://colab.research.google.com/github/mgamzec/GeoPython/blob/main/geemap_sentinel1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geemap

In [ ]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=uH5ZA4VUyjr-RAIf58_D-0nK7OtrGnNzAijBoYl6qAQ&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWgzGCtt8D1-S6pQSr-wetCyYAQIRvzLS7e4gmiAsDUkgRE9i_BfgEU

Successfully saved authorization token.


In [ ]:
import ee
import geemap

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()


In [ ]:
# Load the Sentinel-1 ImageCollection.
#sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate('2015-01-01','2015-03-01')
sentinel1=ee.ImageCollection("COPERNICUS/S1_GRD").filterDate('2016-01-01','2016-05-01')

In [ ]:
vh = sentinel1.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
              .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
              .filter(ee.Filter.eq('instrumentMode', 'IW'))

In [ ]:
# Filter to get images from different look angles.
#vhAscending = vh.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
vhAscending=vh.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
vhDescending = vh.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))

In [ ]:
# Create a composite from means at different polarizations and look angles.
composite = ee.Image.cat([
  vhAscending.select('VH').mean(),
  ee.ImageCollection(vhAscending.select('VV').merge(vhDescending.select('VV'))).mean(),
  vhDescending.select('VH').mean()
]).focal_median()

Visualise

In [ ]:
wdpa_polygon = ee.FeatureCollection('WCMC/WDPA/current/polygons')
# select the pa in cambodia
aoi = wdpa_polygon.filter(ee.Filter.eq('NAME', 'Ang Trapeng Thmor'))


In [ ]:
ffa_db = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD')
                       .filterBounds(aoi)
                       .filterDate(ee.Date('2019-01-01'), ee.Date('2019-07-31'))
                       .median()
                       .clip(aoi))

In [ ]:
# Make an RGB color composite image (VV,VH,VV/VH).
rgb = ee.Image.rgb(ffa_db.select('VV'),
                   ffa_db.select('VH'),
                   ffa_db.select('VV').divide(ffa_db.select('VH')))


In [ ]:
map4 = geemap.Map(center=[12.5657, 105],zoom=5)
map4.add_basemap('SATELLITE')
map4.addLayer(rgb, {'min': [-20, -20, 0], 'max': [0, 0, 2]}, 'sentinel 1')

map4.addLayerControl()
map4

Map(center=[12.5657, 105], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childre…